## Threading und Multiprocessing

- Erstellen Sie ein Python-Programm, das mehrere Threads startet
- Jeder Thread soll Ihren Namen und die ID des Threads auf der Konsole ausgeben 
- Definieren Sie dazu eine Funktion "nachricht_drucken(name)", die fünfmal Ihre Nachricht ausgibt
- Erzeugen Sie drei Threads, die jeweils diese Funktion mit einem eigenen Namen (z. B. „Thread Anke“, „Thread Claire“, „Thread Janosch“) ausführen.
- Starten Sie die Threads.
- Warten Sie mit join() auf das Ende aller Threads.

In [ ]:
import threading
import time

def nachricht_drucken(name):
    for i in range(5):
        thread_id=threading.get_ident()
        print(f"Servus von {name} mit id {thread_id} – Ausgabe {i+1}")
        time.sleep(0.5)  # kleine Pause, um die Ausgabe besser zu sehen

# Threads erstellen
thread_a = threading.Thread(target=nachricht_drucken, args=("Thread Anke",))
thread_b = threading.Thread(target=nachricht_drucken, args=("Thread Claire",))
thread_c = threading.Thread(target=nachricht_drucken, args=("Thread Janosch",))

# Threads starten
thread_a.start()
thread_b.start()
thread_c.start()

# Warten, bis alle Threads fertig sind
thread_a.join()
thread_b.join()
thread_c.join()

print("Alle Threads haben ihre Nachrichten ausgegeben.")

- Simulieren Sie eine Datenbank als gemeinsame Liste database = []
- Erstellen Sie eine Funktion insert_student(name), die einen Eintrag wie {"name": name} zur Liste hinzufügt.
- Erstellen Sie 5 Threads, die jeweils 100 zufällige Namen erzeugen und in die Datenbank schreiben.
- geben Sie am Ende die Anzahl der Einträge in der "Datenbank" aus.




In [ ]:
import threading
import random
import string

database = []

# zufälliges generieren eines Namens
def generate_random_name():
    return ''.join(random.choices(string.ascii_uppercase, k=5))

def insert_student_data(thread_id):
    for _ in range(100):
        name = f"{generate_random_name()}_{thread_id}"
        print(name)
        database.append({"name": name})

threads = []

for i in range(5):  # 5 Threads
    t = threading.Thread(target=insert_student_data, args=(i,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Anzahl der Studierenden in der Datenbank:", len(database))

- Verwenden/implementieren Sie einen Lock, um Race Conditions zu vermeiden
- Diskutieren Sie: Sind Race Conditions hier möglich?

- Erstellen Sie eine Funktion, die in zwei Threads prüft, ob eine Zahl eine Primzahl ist.
- damit das ganze schneller läuft, sollen sich die beiden Threads die Primzahlen teilen.
- Führen Sie die Teilung zunächst so durch, dass sie eine globale liste erstellen und diese Liste vor dem Start teilen
- messen Sie, wie lange das ganze gedauert hat

In [ ]:
import threading
import time

# globale Liste mit Zahlen, die auf primzahligkeit geprüft werden
numbers = [17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 259, 59, 67, 83, 109, 127, 157, 179, 191, 211, 
           241, 277, 288, 283, 331, 390,353, 367, 401, 431, 461, 444,509, 547, 563,900
             ,555,587, 599, 617, 709, 739, 773, 797, 859]
lock = threading.Lock()

# Prime check function prüft, ob Zahl n durch irgedwas teilbar ist 
def is_prime(n):
    if n <= 1:
        return False
    if n > 1:
        # check ob Faktor von irgendwas
        for i in range(2,n):
            if (n % i) == 0: # % ist modulo, sprich rechnet rest aus
                return False
                break
        else:
            return True
        
def worker(sublist):
    thread_id=threading.get_ident()
    for num in sublist:
        erg=is_prime(num)
        time.sleep(.2)
        if erg:
            print(f"thread {thread_id}: {num} is a prime",flush=True)
        else:
            print(f"thread {thread_id}: {num} not a prime",flush=True)


# Split the list into two halves
mid = len(numbers) // 2
teil1 = numbers[:mid]
teil2 = numbers[mid:]


t1 = threading.Thread(target=worker,args=(teil1,))
t2 = threading.Thread(target=worker,args=(teil2,))
start = time.time()
t1.start()
t2.start()

t1.join()
t2.join()
print("Aalle Zahlen überprüft")
end = time.time()

print("Zeit in Sekunden:", end - start)

- Erstellen Sie jetzt dieselbe Funktion nochmal, jedoch so, dass 2 threads abwechselnd auf die Liste "numbers" zugreifen
- sobald eine Zahl geprüft wird, soll diese aus der Liste gelöscht werden
- Hinweis: um zu vermeiden, dass die beidne Threads zeitgleich auf dieselbe zahl zugreifen, muss die Liste gelocked werden, solange einer der threads zugreift.

In [ ]:
import threading
import time

# globale Liste mit Zahlen, die auf primzahligkeit geprüft werden
numbers = [17, 18, 19, 20, 21, 22, 23, 24, 25, 29, 259, 59, 67, 83, 109, 127, 157, 179, 191, 211, 
           241, 277, 288, 283, 331, 390,353, 367, 401, 431, 461, 444,509, 547, 563,900
             ,555,587, 599, 617, 709, 739, 773, 797, 859]
lock = threading.Lock()

# Prime check function prüft, ob Zahl n durch irgedwas teilbar ist 
def is_prime(n):
    time.sleep(.3)
    if n > 1:
       # check ob Faktor von irgendwas
       for i in range(2,n):
           if (n % i) == 0: # % ist modulo, sprich rechnet rest aus
               return False
               break
       else:
           return True
    return True

# Worker function
def worker():
    while True:
        thread_id=threading.get_ident()
        lock.acquire()
        if numbers:
            num = numbers.pop(0)  # entferne oberste nummer aus liste
            lock.release()
            result = is_prime(num)
            if result:
                print(f"thread {thread_id}: {num} is a prime",flush=True)
            else:
                 print(f"thread {thread_id}: {num} not a prime",flush=True)
        else:
            lock.release()
            break

t1 = threading.Thread(target=worker)
t2 = threading.Thread(target=worker)
start=time.time()
t1.start()
t2.start()

t1.join()
t2.join()
print("alles geprüft")
end = time.time()
print("Zeit in Sekunden:", end - start)


### Multiprocessing vs. Multithreading

Sie sollen berechnen, wieviel Strom eine Familie aus ihrer PV-Anlage selbst nutzen kann und wieviel sie verschenkt. Die dazu erforderlichen Daten liegen in einer nervigen Form vor, daher sollen Sie diese zunächst in eine Datenbank schreiben und anschließend wieder auslesen, um die Nutzun zu berechnen...

- in moodle unter vl9..\data liegen Daten für die Erzeugung von Strom aus einer PV-Anlage und für die Stromnachfrage eines Haushalts. Die Daten liegen in csv-Dateien in stündlicher Auflösung tageweise vor (1 Datei pro Tag sowohl für die PV-Anlage als auch die Stromnachfrage)
- das ist lästig, lesen Sie daher die csv-Dateien ein. Damit das schneller geht, erstellen Sie Threads, die sich die Dateien aufteilen
    - thread 1 übernimmt die PV-Daten
    - Thread 2 übernimmt die Lastdaten
    - bauen Sie einen Sleep-Timer ein, damit es interessanter wird
- schreiben Sie die die tageweisen Daten in zwei Tabellen (pv bzw. last) in ihrer mysql-Datenbank aus der dritten Übung. Hinweis: Verwenden Sie für die Benennung des Tables in der Mysql-Datenbank nicht einfach nur das Wort "load", dieses ist vorbelegt. Stattdessen last


In [ ]:
import pandas as pd
import os
import sqlalchemy 

import threading
import time
os.chdir("C:\\Users\\kuehnbam\\Nextcloud\\prog2\\vl9")

user = "root"
password = ""
host = "localhost"
port = 3306
database = "prog2"

# Verbindungs-URL zusammenbauen
db_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
# Neue Engine im Prozess erstellen
engine = sqlalchemy.create_engine(db_url)
# Pfad zum Ordner mit CSV-Dateien
def csv_aus_ordner_lesen(folderpath,type):
    # Durchlaufe alle Dateien im Ordner
    for dateiname in os.listdir(folderpath):
        #time.sleep(0.2)
        if dateiname.endswith(".csv"):
            pfad_zur_datei = os.path.join(folderpath, dateiname)
            df = pd.read_csv(pfad_zur_datei,sep=";",encoding='utf-8', decimal='.')
            print(f"{dateiname} geladen mit {len(df)} Zeilen.")
            df.to_sql(type, con=engine, if_exists='append', index=False)
            print(f"{dateiname} in tabelle {type} geschrieben")

            #daten.append(df)

# Threads erzeugen
thread1 = threading.Thread(target=csv_aus_ordner_lesen, args=("data\\pv", "pv", ))
thread2 = threading.Thread(target=csv_aus_ordner_lesen, args=("data\\last", "last", ))

# Threads starten
start=time.time()
thread1.start()
thread2.start()

# Auf Threads warten
thread1.join()
thread2.join()
end=time.time()
print("Zeit in Sekunden:", end - start)
print('end of file')



- Wenn das durch ist, lesen Sie die Daten im Ganzen wieder aus der Mysql aus und berechnen Sie, ob die Familie mit der PV-Anlage bilanziell einen Stromüberschuss generiert hat, indem Sie die Lastsumme von der PV-Summe abziehen

In [ ]:
query= "SELECT * FROM pv"
dt_pv=pd.read_sql(query,con=engine)
query= "SELECT * FROM last"
dt_last=pd.read_sql(query,con=engine)
dt_pv=dt_pv.loc[0:8759,:]
dt_last=dt_last.loc[0:8759,:]

pv_sum=dt_pv['pv_generation'].sum()
last_sum=dt_last['electricity_consumption'].sum()

bilanz=pv_sum-last_sum
print("Der jährliche PV-Überschuss des Haushalts beträgt ",bilanz," kWh")

- Erstellen Sie das Lesen der Dateien und das schreiben in die DB als echt paralleles Programm mit Multiprocessing mit 2 Kernen
- vergleichen Sie, was schneller war
- Hinweis: Das multiprocessing-Programm müssen Sie vermutlich außerhalb von Jupyter-Notebook in einer .py-Datei ausführen


In [ ]:
import os
import pandas as pd
import sqlalchemy
from multiprocessing import Process
import time

def csv_aus_ordner_lesen(folderpath, table_name):
    user = "root"
    password = ""
    host = "localhost"
    port = 3306
    database = "prog2"

    # Verbindungs-URL zusammenbauen
    db_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
    # Neue Engine im Prozess erstellen
    engine = sqlalchemy.create_engine(db_url)

    for dateiname in os.listdir(folderpath):
        if dateiname.endswith(".csv"):
            pfad_zur_datei = os.path.join(folderpath, dateiname)
            df = pd.read_csv(pfad_zur_datei, sep=";", encoding='utf-8', decimal='.')
            print(f"{dateiname} geladen mit {len(df)} Zeilen.")
            df.to_sql(table_name, con=engine, if_exists='append', index=False)
            print(f"{dateiname} in Tabelle {table_name} geschrieben")

if __name__ == "__main__":
    os.chdir("C:\\Users\\kuehnbam\\Nextcloud\\prog2\\vl9")
    start=time.time()
    # Prozesse erzeugen
    p1 = Process(target=csv_aus_ordner_lesen, args=("data\\pv", "pv"))
    p2 = Process(target=csv_aus_ordner_lesen, args=("data\\last", "last"))

    # Prozesse starten
    p1.start()
    p2.start()

    # Auf Prozesse warten
    p1.join()
    p2.join()
    end=time.time()
    print("Zeit in Sekunden:", end - start)

    print("End of file")

- Diskutieren Sie, auf welche weise Sie diesen Prozesse noch verbessern können, wenn wirklich große Datenmengen ins Spiel kommen
- Testen Sie den Prozess auch noch mit einer Sqlite-Datei anstatt Ihrer mySql-Datenbank